In [ ]:
import h5py
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelBinarizer,OneHotEncoder
import pickle 

PATH_TO_HDF5_FILE = "./data/demos/logmel_demos.hdf5"

def scale_minmax(X, min=0.0, max=1.0):
    X_std = (X - X.min()) / (X.max() - X.min())
    X_scaled = X_std * (max - min) + min
    return X_scaled

with h5py.File(PATH_TO_HDF5_FILE, "r") as f:
    a_group_key = list(f.keys())
    data = pd.DataFrame()
    for key in a_group_key:
        data[key] = np.array(f[key]).tolist()

for column in data.columns:
    if(column != 'logmel'):
       data[column] = data[column].apply(lambda x: x.decode('UTF-8'))

speakerIDs = [22, 57, 1, 36, 55, 18, 6, 33, 23, 25, 68, 51, 7, 50, 54, 48, 49, 47, 43, 40, 17, 60, 21, 63, 4, 26, 9, 32,
                 42, 53, 58, 11, 2, 14, 20, 34, 62, 65, 13, 39, 19,56, 37, 44, 29, 45, 38, 31, 46, 61, 41, 27,52, 12, 64,
                30, 15, 10, 28, 24, 69, 8, 16,59, 5, 3, 66, 67]




In [ ]:
for column in data.columns:
    if(column != 'logmel'):
        data[column] = data[column].apply(lambda x: x.decode('UTF-8'))

In [ ]:
import math
split_index = math.floor(data.shape[0]*1/10)
evaluationData = data[:split_index]
data = data[split_index:]
#check for class distribution
#dist_ = evaluationData['emotion'].value_counts()
#print(dist_)
#evaluationData = data[split_index:]
#print(dist_end)

In [ ]:
data['speaker_id'].value_counts()

In [ ]:
speakerIDs = [22, 57, 1, 36, 55, 18, 6, 33, 23, 25, 68, 51, 7, 50, 54, 48, 49, 47, 43, 40, 17, 60, 21, 63, 4, 26, 9, 32,
                 42, 53, 58, 11, 2, 14, 20, 34, 62, 65, 13, 39, 19,56, 37, 44, 29, 45, 38, 31, 46, 61, 41, 27,52, 12, 64,
                30, 15, 10, 28, 24, 69, 8, 16,59, 5, 3, 66, 67]

In [ ]:
#clean evaluation data
dataDict = {}
x_eval = evaluationData.logmel
x_eval = np.array([x for x in x_eval.values  ])
x_eval = scale_minmax(x_eval)
x_eval = x_eval[:,:,:,np.newaxis]
y_eval = evaluationData.emotion
y_eval = np.array([x for x in y_eval.values  ])
dataDict["x_eval"] = x_eval
dataDict["y_eval"] = y_eval

In [ ]:
import math
n_clients = 68
groups = np.array(np.array_split(speakerIDs,n_clients))
groups = np.array([v for v in groups ])
i=1
for group in groups:
    x_train = np.array([])
    y_train = np.array([])
    for speakerID in group:
        train_data = data[data["speaker_id"]==speakerID]
        #_train
        print(_x_train)
        _x_train = train_data.logmel
        _x_train = np.array([v for v in _x_train.values ])
        _x_train = scale_minmax(_x_train)
        _x_train = _x_train[:,:,:,np.newaxis]
        _y_train = train_data.emotion
        _y_train = np.array([v for v in _y_train.values ])
        if  (x_train.size == 0):
            x_train = _x_train
            y_train = _y_train
        else:
            x_train = np.concatenate((x_train, _x_train), axis=0)
            y_train = np.concatenate((y_train, _y_train), axis=0)
    dataDict[f"client_{i}"] = {
        "x_train": x_train,
        "y_train": y_train,
    }
    i = i+1


In [ ]:
with open(f"demos_{n_clients}clients_data.pickle", "wb") as handle:
    pickle.dump(dataDict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
evaluationData['emotion'].value_counts()